# Tratamento de dados faltantes

In [11]:
import pandas as pd 
dados = pd.read_csv('aluguel_residencial.csv', sep = ';')
dados.head(10)

,Unnamed: 0,Tipo,Bairro,Quartos,Vagas,Suites,Area,Valor,Condominio,IPTU
0,0,Quitinete,Copacabana,1.0,0.0,0.0,40.0,1700.0,500.0,60.0
1,1,Casa,Jardim Botânico,2.0,0.0,1.0,100.0,7000.0,NaN,NaN
2,2,Apartamento,Centro,1.0,0.0,0.0,15.0,800.0,390.0,20.0
3,3,Apartamento,Higienópolis,1.0,0.0,0.0,48.0,800.0,230.0,NaN
4,4,Apartamento,Vista Alegre,3.0,1.0,0.0,70.0,1200.0,NaN,NaN
5,5,Apartamento,Cachambi,2.0,0.0,0.0,50.0,1300.0,301.0,17.0
6,6,Casa de Condomínio,Barra da Tijuca,5.0,4.0,5.0,750.0,22000.0,NaN,NaN
7,7,Casa de Condomínio,Ramos,2.0,2.0,0.0,65.0,1000.0,NaN,NaN
8,8,Apartamento,Centro,1.0,0.0,0.0,36.0,1200.0,NaN,NaN
9,9,Apartamento,Grajaú,2.0,1.0,0.0,70.0,1500.0,642.0,74.0


Perceberemos rapidamente que as variáveis Condominio e IPTU apresentam dados faltantes, anotados com o termo NaN. Não podemos analisar o conjunto de dados inteiro a olho nu, precisamos de um método que nos auxilie nessa tarefa. 

In [12]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26635 entries, 0 to 26634
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  26635 non-null  int64  
 1   Tipo        26635 non-null  object 
 2   Bairro      26635 non-null  object 
 3   Quartos     26635 non-null  float64
 4   Vagas       26635 non-null  float64
 5   Suites      26635 non-null  float64
 6   Area        26634 non-null  float64
 7   Valor       26625 non-null  float64
 8   Condominio  24474 non-null  float64
 9   IPTU        18584 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 2.0+ MB


Iremos observar as variáveis que apresentam dados faltantes. Seguiremos o seguinte modelo: 

In [13]:
dados['Valor'].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
26630    False
26631    False
26632    False
26633    False
26634    False
Name: Valor, Length: 26635, dtype: bool

Teremos como retorno uma Series booleana, em que o valor True marca os elementos nulos.

In [14]:
dados[dados['Valor'].isnull()] #separamos todos os dados que possuem valor NaN

,Unnamed: 0,Tipo,Bairro,Quartos,Vagas,Suites,Area,Valor,Condominio,IPTU
58,58,Apartamento,Barra da Tijuca,2.0,1.0,1.0,70.0,NaN,970.0,68.0
1492,1492,Apartamento,Leme,2.0,0.0,0.0,75.0,NaN,878.0,NaN
1683,1683,Casa,Campo Grande,3.0,4.0,3.0,363.0,NaN,NaN,NaN
2012,2012,Apartamento,Botafogo,2.0,0.0,0.0,95.0,NaN,1010.0,170.0
2034,2034,Apartamento,Copacabana,2.0,0.0,0.0,72.0,NaN,850.0,NaN
4941,4941,Casa,Campo Grande,3.0,2.0,1.0,100.0,NaN,NaN,NaN
8568,8568,Apartamento,Leme,2.0,0.0,1.0,75.0,NaN,878.0,NaN
8947,8947,Apartamento,Glória,3.0,0.0,1.0,135.0,NaN,910.0,228.0
9149,9149,Apartamento,Gávea,3.0,1.0,1.0,105.0,NaN,880.0,221.0
22580,3,2,420,8000.0,3400.0,1400.0,NaN,NaN,NaN,NaN


A principal informação em nosso banco de dados são os valores dos aluguéis, portanto não faz sentido que tenhamos dados nulos neste campo. Precisamos elimina-los, e para tanto, usaremos o método dropna(). 
Para métodos de comparação, faremos o seguinte:

Criaremos uma variável A, que abrigara dados, isto é, nosso DataFrame original. Em seguida, escreveremos dados.dropna(), o método receberá como argumento susbset, que se trata de uma lista de variáveis. Em seguida, criaremos uma nova variável B, que abrigará o resultado do novo conjunto de dados que não contém valores nulos. Então, escreveremos A - B.

In [15]:
A = dados.shape[0]
dados.dropna(subset = ['Valor'], inplace = True)
B = dados.shape[0]
print(A - B) #Eliminamos as 9 entradas que tinham o Valor como NaN, agora precisamos fazer isso para todas as outras variáveis.

10


Temos as variáveis Condomínio e IPTU com alguns problemas. No caso de Condominio, os dados nulos nem sempre são incorretos, afinal no caso de uma casa, realmente não há esse tipo de tarifa. Para este caso, devemos considerar a variável Tipo: caso o imóvel seja um apartamento, os dados nulos serão excluídos.

In [16]:
dados[dados['Condominio'].isnull()].shape[0]

2158

In [17]:
selecao = (dados['Tipo'] == 'Apartamento') & (dados['Condominio'].isnull())

In [18]:
A = dados.shape[0]
dados = dados[~selecao]
B = dados.shape[0]
print(A - B)

873


 Vamos verificar quantas assinaturas nulas temos em Condominio agora: 

In [19]:
dados[dados['Condominio'].isnull()].shape[0]

1285

In [20]:
selecao = (dados['Tipo'] == 'Apartamento') & (dados['Condominio'].isnull())

O que faremos é manter esses dados, mas atribuir o valor 0 a eles. Temos uma função para esta ação fillna(), que receberá o valor 0 e inplance = True.

In [21]:
dados.fillna(0, inplace = True)

In [22]:
dados.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25752 entries, 0 to 26634
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  25752 non-null  int64  
 1   Tipo        25752 non-null  object 
 2   Bairro      25752 non-null  object 
 3   Quartos     25752 non-null  float64
 4   Vagas       25752 non-null  float64
 5   Suites      25752 non-null  float64
 6   Area        25752 non-null  float64
 7   Valor       25752 non-null  float64
 8   Condominio  25752 non-null  float64
 9   IPTU        25752 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 2.2+ MB


# Exercício

In [27]:
imoveis = pd.DataFrame([['Apartamento', None, 970, 68], 
                        ['Apartamento', 2000, 878, 112], 
                        ['Casa', 5000, None, 500], 
                        ['Apartamento', None, 1010, 170], 
                        ['Apartamento', 1500, 850, None], 
                        ['Casa', None, None, None], 
                        ['Apartamento', 2000, 878, None], 
                        ['Apartamento', 1550, None, 228], 
                        ['Apartamento', 2500, 880, 195]], 
                        columns = ['Tipo', 'Valor', 'Condominio', 'IPTU'])

In [28]:
imoveis.dropna(subset = ['Valor'], inplace = True)
imoveis

,Tipo,Valor,Condominio,IPTU
1,Apartamento,2000.0,878.0,112.0
2,Casa,5000.0,NaN,500.0
4,Apartamento,1500.0,850.0,NaN
6,Apartamento,2000.0,878.0,NaN
7,Apartamento,1550.0,NaN,228.0
8,Apartamento,2500.0,880.0,195.0


In [29]:
selecao = (imoveis['Tipo'] == 'Apartamento') & (imoveis['Condominio'].isnull())
imoveis = imoveis[~selecao]
imoveis

,Tipo,Valor,Condominio,IPTU
1,Apartamento,2000.0,878.0,112.0
2,Casa,5000.0,NaN,500.0
4,Apartamento,1500.0,850.0,NaN
6,Apartamento,2000.0,878.0,NaN
8,Apartamento,2500.0,880.0,195.0


In [30]:
imoveis = imoveis.fillna({'Condominio': 0, 'IPTU': 0})
imoveis

,Tipo,Valor,Condominio,IPTU
1,Apartamento,2000.0,878.0,112.0
2,Casa,5000.0,0.0,500.0
4,Apartamento,1500.0,850.0,0.0
6,Apartamento,2000.0,878.0,0.0
8,Apartamento,2500.0,880.0,195.0


In [31]:
imoveis.index = range(imoveis.shape[0])
imoveis

,Tipo,Valor,Condominio,IPTU
0,Apartamento,2000.0,878.0,112.0
1,Casa,5000.0,0.0,500.0
2,Apartamento,1500.0,850.0,0.0
3,Apartamento,2000.0,878.0,0.0
4,Apartamento,2500.0,880.0,195.0
